In [20]:
band_name = 'green day'
Song_List = []
with open('SongList_GreenDay.csv','r') as infile:
    Song_List = infile.readlines()
for i in range(len(Song_List)):
    Song_List[i] = Song_List[i].replace('\n','')

In [26]:
import os
if not os.path.exists(band_name) : os.mkdir(band_name)
os.chdir('/Users/syushengwei/Desktop/Song_Crawler/'+band_name)

In [28]:
os.getcwd()

'/Users/syushengwei/Desktop/Song_Crawler/green day'

In [29]:
##reference from :https://www.quora.com/Whats-a-good-api-to-use-to-get-song-lyrics
##programmer : Sagun Shrestha
import re
import urllib.request
from bs4 import BeautifulSoup
 
def get_lyrics(artist,song_title):
    artist = artist.lower()
    song_title = song_title.lower()
    # remove all except alphanumeric characters from artist and song_title
    artist = re.sub('[^A-Za-z0-9]+', "", artist)
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)
    if artist.startswith("the"):    # remove starting 'the' from artist e.g. the who -> who
        artist = artist[3:]
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"
    
    try:
        content = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(content, 'html.parser')
        lyrics = str(soup)
        # lyrics lies between up_partition and down_partition
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->'
        down_partition = '<!-- MxM banner -->'
        lyrics = lyrics.split(up_partition)[1]
        lyrics = lyrics.split(down_partition)[0]
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').strip()
        return lyrics
    except Exception as e:
        return "Exception occurred \n" +str(e)

In [72]:
#crawl song lyric
for title in Song_List:
    Song_Lyric = get_lyrics(band_name,title)
    if "Exception occurred" in Song_Lyric:
        continue
    #form lyric into     
    Song_Lyric = Song_Lyric.split('<br/>\r\n')
    for i in range(len(Song_Lyric)):
        if '\n' in Song_Lyric[i] : Song_Lyric[i] = Song_Lyric[i].replace('\n','')
        if '<br/>' in Song_Lyric[i] : Song_Lyric[i] = Song_Lyric[i].replace('<br/>','')
    Song_Lyric = '\n'.join(Song_Lyric)
    with open(title+'.txt','w') as outfile:
        outfile.write(band_name)
        outfile.write('\n')
        outfile.write(Song_Lyric)
    

In [62]:
#crawl song tags
import pylast
API_KEY = "761ed93bfcab1f36f93fb293e9971abb"
API_SECRET = "aad8d63d9790603f88b7857b0809dce5"
username =      input("Username for last.fm : ")
password_hash = pylast.md5(getpass.getpass("Password for last.fm : "))
network = pylast.LastFMNetwork(api_key = API_KEY, 
                               api_secret = API_SECRET, 
                               username = username, 
                               password_hash = password_hash)

Username for last.fm : feather8405
Password for last.fm : ········


In [65]:
print(len(Song_List))

254


In [69]:
Band_Tags = {}
Band_Song = {'green day':Song_List}
for band_name in Band_Song:
    Song_Tags = {}
    for song_name in Band_Song[band_name]:
        print(band_name,' : ',song_name)
        try: 
            Last_song = network.get_track(band_name, song_name)
        except: 
            print("can't find song : ", song_name)
            continue
        try:
            Last_Tags = Last_song.get_top_tags()
            A_Song_Tags = []
            for tag in Last_Tags:
                A_Song_Tags.append(tag.item.get_name())
        except:
            print("can't find tags : ", song_name)
            continue
        Song_Tags.update({song_name : A_Song_Tags})
    Band_Tags.update({band_name : Song_Tags})

green day  :  1,000 Hours
green day  :  16
green day  :  19th Nervous Breakdown
green day  :  2000 Light Years Away
green day  :  21 Guns
green day  :  21st Century Breakdown
green day  :  409 in Your Coffeemaker
green day  :  80
green day  :  86
green day  :  8th Avenue Serenade
green day  :  99 Revolutions
green day  :  Alfred E. Normal
can't find tags :  Alfred E. Normal
green day  :  Alison
green day  :  All the Time
green day  :  Amanda
green day  :  American Eulogy
green day  :  American Idiot
green day  :  Amy
green day  :  Android
green day  :  Angel
green day  :  Angel Blue
green day  :  Another State of Mind
green day  :  Are We the Waiting
green day  :  Armatage Shanks
green day  :  Ashley
green day  :  At the Library
green day  :  Bab's Uvula Who?
green day  :  Baby Blue
green day  :  Baby Eyes
green day  :  Back In the USA
green day  :  The Ballad of Wilhelm Fink
green day  :  Bang Bang
green day  :  Basket Case
green day  :  Before the Lobotomy
green day  :  Best Thing in

In [70]:
Band_Tags

{'green day': {'1,000 Hours': ['punk rock',
   'punk',
   'rock',
   'green day',
   'alternative rock',
   'favorite songs',
   'Grunge',
   'alternative',
   'beautiful',
   '90s',
   'pop punk',
   'Californian Punk',
   'jazz',
   '80s',
   'american',
   'cool',
   'male vocalists',
   'favorite band',
   'my favorite songs',
   'sacredsummers',
   'songs i wish were written for me',
   'fav songs',
   'Old School Green Day',
   'Zodo',
   'lyrics matter',
   'To juz klasyka',
   'fpbdml gif'],
  '16': ['punk rock',
   'punk',
   'rock',
   'green day',
   '90s',
   'alternative',
   'alternative rock',
   'favorite songs',
   'Californian Punk',
   'chillout',
   'Grunge',
   'friends',
   'Old School Green Day',
   '80s',
   'pop punk',
   'male vocalists',
   'FUCKING AWESOME',
   'dream means california',
   'jazz',
   'hard rock',
   'favorites',
   'memories',
   'Pop-punk',
   'gute laune',
   'number',
   '16',
   'My Favorites',
   'grew up on',
   'gd',
   '500 essential

In [71]:
import json
J_tags = json.dumps(Band_Tags,ensure_ascii=False)
import codecs
with codecs.open('GreenDay_Tags.json','w') as outfile:
    outfile.write(J_tags)